### Collate XLSSC122 and 3DHST galfit catalogs
- Dec 2/2020

### Load modules

In [27]:
from __future__ import division
import sys
sys.path.insert(0,'/data/emiln/XLSSC122_GalPops/Analysis/Modules')
from GalfitPyWrap import galfitwrap as gf
import galfit_helpers as gfh
import os
import numpy as np
import pandas as pd
import glob
from astropy import units as u
from astropy import wcs
from astropy.coordinates import SkyCoord
from astropy.io import fits, ascii
from astropy.table import Table, hstack, join
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### HST

In [10]:
# Load catalogs
galfit_filename = '/data/emiln/XLSSC122_GalPops/Analysis/HST/Results/final_galfit_stellar_PSF.csv'
phot_filename =  '/data/emiln/XLSSC122_GalPops/Data/HST/Products/catalogs/xlssc122_F105_F140_redshifts_short_v4_snr_em_member2_by_hand.cat'
galfit_df = pd.read_csv(galfit_filename)
phot_df = pd.read_csv(phot_filename, delim_whitespace=True)

In [23]:
# Merge
cols_to_use = phot_df.columns.difference(galfit_df.columns)
hst_mdf = galfit_df.merge(phot_df[cols_to_use],how='inner',left_on='ID',right_on='phot-id')
print(hst_mdf.columns.values)
print(len(hst_mdf))
print(len(galfit_df))

['ID' 'ra' 'dec' 're' 're_err' 'ar' 'ar_err' 'n' 'n_err' 'mag' 'mag_err'
 'sky' 'sky_err' 'chi2nu' 'ErrorValue' 'asec_fwhm' 'chi' 'em-code'
 'f105_0p8' 'f105_e0p8' 'f105_ekron' 'f105_kron' 'f140_0p8' 'f140_e0p8'
 'f140_ekron' 'f140_kron' 'mcode' 'member1' 'member2' 'phot-id' 'snr'
 'spec-id' 'zhi' 'zlo' 'zz']
37
37


In [25]:
# Save
hst_savename = '/data/emiln/XLSSC122_GalPops/Analysis/Results/hst_galfit_full.csv'
hst_mdf.to_csv(hst_savename,index=False)

### 3DHST

In [37]:
# Load catalogs
DHST_cat_root = '/data/emiln/XLSSC122_GalPops/Data/3DHST/Products/catalogs/'
DHST_cat_filename = DHST_cat_root+'cosmos_3dhst.v4.1.cats/Catalog/cosmos_3dhst.v4.1.cat'
phot_df = gfh.load_3DHST_galaxy_catalog(DHST_cat_filename,  mag='F140W', magthresh=24, z=2.00, z_thresh=0.1, overwrite=False, verbose=True, goodfit=True) 

galfit_filename = '/data/emiln/XLSSC122_GalPops/Analysis/3DHST/Results/3DHST_z2_full_data_cps_sigma_rms_meanexp_cps_w5HLR_F140W_PSF_5min_CONV64_CONSTR_DYNMAG.csv'
galfit_df = pd.read_csv(galfit_filename)


Checking if catalog exists at /data/emiln/XLSSC122_GalPops/Data/3DHST/Products/catalogs/magthresh24_z2.0_zthresh0.1.csv
Catalog already exists, returning DataFrame


In [41]:
phot_df.columns.values
galfit_df.columns.values

array(['ID', 'ra', 'dec', 're', 're_err', 'ar', 'ar_err', 'n', 'n_err',
       'mag', 'mag_err', 'sky', 'sky_err', 'chi2nu', 'ErrorValue'],
      dtype=object)

In [46]:
# Merge
# cols_to_use = phot_df.columns.difference(galfit_df.columns)
dhst_mdf = galfit_df.merge(phot_df,how='inner',on='ID',suffixes=('','_y'))
dhst_mdf.drop(labels=['ra_y','dec_y'],axis=1,inplace=True)
print(dhst_mdf.columns.values)
print(len(dhst_mdf))
print(len(galfit_df))

['ID' 'ra' 'dec' 're' 're_err' 'ar' 'ar_err' 'n' 'n_err' 'mag' 'mag_err'
 'sky' 'sky_err' 'chi2nu' 'ErrorValue' 'id' 'x' 'y' 'faper_F160W'
 'eaper_F160W' 'faper_F140W' 'eaper_F140W' 'f_F160W' 'e_F160W' 'w_F160W'
 'f_U' 'e_U' 'w_U' 'f_B' 'e_B' 'w_B' 'f_G' 'e_G' 'w_G' 'f_V' 'e_V' 'w_V'
 'f_F606W' 'e_F606W' 'w_F606W' 'f_R' 'e_R' 'w_R' 'f_Rp' 'e_Rp' 'w_Rp'
 'f_I' 'e_I' 'w_I' 'f_Ip' 'e_Ip' 'w_Ip' 'f_F814W' 'e_F814W' 'w_F814W'
 'f_Z' 'e_Z' 'w_Z' 'f_Zp' 'e_Zp' 'w_Zp' 'f_UVISTA_Y' 'e_UVISTA_Y'
 'w_UVISTA_Y' 'f_F125W' 'e_F125W' 'w_F125W' 'f_J1' 'e_J1' 'w_J1' 'f_J2'
 'e_J2' 'w_J2' 'f_J3' 'e_J3' 'w_J3' 'f_J' 'e_J' 'w_J' 'f_UVISTA_J'
 'e_UVISTA_J' 'w_UVISTA_J' 'f_F140W' 'e_F140W' 'w_F140W' 'f_H1' 'e_H1'
 'w_H1' 'f_H2' 'e_H2' 'w_H2' 'f_H' 'e_H' 'w_H' 'f_UVISTA_H' 'e_UVISTA_H'
 'w_UVISTA_H' 'f_K' 'e_K' 'w_K' 'f_Ks' 'e_Ks' 'w_Ks' 'f_UVISTA_Ks'
 'e_UVISTA_Ks' 'w_UVISTA_Ks' 'f_IRAC1' 'e_IRAC1' 'w_IRAC1' 'f_IRAC2'
 'e_IRAC2' 'w_IRAC2' 'f_IRAC3' 'e_IRAC3' 'w_IRAC3' 'f_IRAC4' 'e_IRAC4'
 'w_IRAC4' 'f_IA427

In [47]:
# Save
dhst_savename = '/data/emiln/XLSSC122_GalPops/Analysis/Results/3dhst_galfit_full.csv'
dhst_mdf.to_csv(dhst_savename,index=False)